# MLDL2 Homework 3

In [ ]:
import os
os.environ["OMP_NUM_THREADS"]= '4'
os.environ["OMP_THREAD_LIMIT"] = '4'
os.environ["MKL_NUM_THREADS"] = '4'
os.environ["NUMEXPR_NUM_THREADS"] = '4'
os.environ["OMP_NUM_THREADS"] = '4'
os.environ["PAPERLESS_AVX2_AVAILABLE"]="false"
os.environ["OCR_THREADS"] = '4'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn.functional as F
import time
import os
import math
from tqdm import tqdm
from collections import OrderedDict

from copy import deepcopy
from datetime import datetime

import logging
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset
from shutil import copyfile

import random
random.seed(42)


import warnings
warnings.filterwarnings(action='ignore')

# 1. Load the CIFAR-10, CIFAR-100 Datasets

In [ ]:
BATCH_SIZE = 256

cifar10_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.491, 0.482, 0.447), (0.247, 0.244, 0.262))
])

cifar100_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276))
])

data_dir = "./data/"
os.makedirs(data_dir, exist_ok=True)

cifar10_train_dataset = datasets.CIFAR10(
    root=data_dir,
    train=True,
    download=not os.path.exists(os.path.join(data_dir, 'cifar-10-batches-py')),
    transform=cifar10_transform
)

cifar100_train_dataset = datasets.CIFAR100(
    root=data_dir,
    train=True,
    download=not os.path.exists(os.path.join(data_dir, 'cifar-100-python')),
    transform=cifar100_transform
)

# Create DataLoaders
cifar10_train_loader = torch.utils.data.DataLoader(
    dataset=cifar10_train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

cifar100_train_loader = torch.utils.data.DataLoader(
    dataset=cifar100_train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)

# 2. Load the Caltech UCSD Birds-200 Dataset

In [ ]:
class CUB_Dataset(Dataset):
    def __init__(self,img_file, label_file, batch_size, transform=None):
        self.img = np.load(img_file)
        self.labels = np.load(label_file)
        self.transform = transform
        self.batch_size = batch_size

    def __len__(self):
        return len(self.img)

    def __getitem__(self, idx):
        image = self.img[idx]
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]

        return image,label

cub_bird_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

cub_train_dataset = CUB_Dataset(img_file="./CUB_train_images.npy",
                                        label_file="./CUB_train_labels.npy", batch_size = 64, transform=cub_bird_transform)

cub_train_loader = torch.utils.data.DataLoader(cub_train_dataset, batch_size= 64, shuffle=True)

cub_val_dataset = CUB_Dataset(img_file="./CUB_val_images.npy",
                                        label_file="./CUB_val_labels.npy", batch_size = 64, transform=cub_bird_transform)
cub_val_loader = torch.utils.data.DataLoader(cub_val_dataset, batch_size = 64, shuffle=False)

In [ ]:
# Number of samples in the dataset

print("cifar10 train dataset size : ", len(cifar10_train_dataset))
print("cifar100 train dataset size : ", len(cifar100_train_dataset))
print("caltech bird train dataset size : ", len(cub_train_dataset))
print("caltech bird validation dataset size : ", len(cub_val_dataset))

## CIFAR-10 Visualiztion

In [ ]:
# Plot the training images and labels

cifar10_denormalize = transforms.Normalize(mean=[-0.491/0.247, -0.482/0.244, -0.447/0.262], std=[1/0.247, 1/0.244, 1/0.262])
to_pil_image = transforms.functional.to_pil_image

images, labels = next(iter(cifar10_train_loader))

fig, ax = plt.subplots(1, 4, figsize=(16, 4))
ax[0].imshow(to_pil_image(cifar10_denormalize(images[0])))
ax[1].imshow(to_pil_image(cifar10_denormalize(images[1])))
ax[2].imshow(to_pil_image(cifar10_denormalize(images[2])))
ax[3].imshow(to_pil_image(cifar10_denormalize(images[3])))
plt.show()

print(labels[:4])

## CIFAR-100 Visualiztion

In [ ]:
# Plot the training images and labels

cifar100_denormalize = transforms.Normalize(mean=[-0.507/0.267, -0.487/0.256, -0.441/0.276], std=[1/0.267, 1/0.256, 1/0.276])
to_pil_image = transforms.functional.to_pil_image

images, labels = next(iter(cifar100_train_loader))

fig, ax = plt.subplots(1, 4, figsize=(16, 4))
ax[0].imshow(to_pil_image(cifar100_denormalize(images[0])))
ax[1].imshow(to_pil_image(cifar100_denormalize(images[1])))
ax[2].imshow(to_pil_image(cifar100_denormalize(images[2])))
ax[3].imshow(to_pil_image(cifar100_denormalize(images[3])))
plt.show()

print(labels[:4])

## Caltech UCSD Birds-200 Visualiztion

In [ ]:
# Plot the training images and labels

cub_denormalize = transforms.Normalize(mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225], std=[1/0.229, 1/0.224, 1/0.225])
to_pil_image = transforms.functional.to_pil_image

images, labels = next(iter(cub_train_loader))

fig, ax = plt.subplots(1, 4, figsize=(16, 4))
ax[0].imshow(to_pil_image(cub_denormalize(images[0])))
ax[1].imshow(to_pil_image(cub_denormalize(images[1])))
ax[2].imshow(to_pil_image(cub_denormalize(images[2])))
ax[3].imshow(to_pil_image(cub_denormalize(images[3])))
plt.show()

print(labels[:4])

# 3. Define the Model Architecture

Here we define the model. Below is very simple model with CNN. You can customize your own model and note that you are not limited to use any methods. **But you are not allowed to use pretrained weight**

In [ ]:
concat = lambda x: np.concatenate(x, axis=0)
to_np = lambda x: x.data.to("cpu").numpy()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class RunningParams:
    def __init__(self, training_type=None, version=18, seed=None):
        # Use the global training_type if not specified
        if training_type is None:
            training_type = 'CUB'

        # # The seed is fixed to ensure reproducibility.
        if seed is not None:
            self.set_seed(seed)

        self.set_active_training(training_type)

        self.TRANSFORMER_ARCH = True
        self.VisionTransformer = False
        self.resnet = version

        self.k_value = 1

        # Retrieving NNs and sample positive and negative pairs
        # Set it when you extract the NNs. data_dir is the folder containing query images for KNN retrieval
        # Set it when you run train/test
        self.set = 'train'
        self.PRODUCT_OF_EXPERTS = True

        self.parent_dir = '/home/stateun/'
        self.prj_dir = '/home/stateun/MLDL2/'
        self.model_dir = f'{self.prj_dir}/pretrained_models'

        # General
        self.conv_layer = 4

        self.feat_map_size = {self.conv_layer: 49}  # 7x7
        if self.resnet == 50:
            self.conv_layer_size = {4: 2048}
        elif self.resnet == 34:
            self.conv_layer_size = {4: 512}
        elif self.resnet == 18:
            self.conv_layer_size = {4: 512}
        else:
            print('Not supported architecture! Exiting...')
            exit(-1)

        # Visualization
        self.VISUALIZE_COMPARATOR_CORRECTNESS = False
        self.VISUALIZE_COMPARATOR_HEATMAPS = False

        # Set training-specific parameters
        self.set_training_params()

    def set_seed(self, seed):
        """
        Set the random seed for reproducibility.
        """
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        print(f"Seed set to: {seed}")

    def set_active_training(self, training_type):
        # Set all training modes to False
        self.CUB_TRAINING = False

        # Activate the chosen training mode
        if training_type == 'CUB':
            self.CUB_TRAINING = True
        else:
            raise ValueError("Invalid training type. Please choose 'CUB', 'CIFAR-10', or 'CIFAR-100'.")

    def set_training_params(self):
        # Set parameters specific to the active training mode
        if self.CUB_TRAINING:
            # Set parameters for CUB training
            self.train_path = 'MLDL2'
            self.test_path = 'datasets/CUB/test0'
            self.combined_path = 'datasets/CUB/combined'
            self.batch_size = 64
            self.epochs = 100

            self.learning_rate = 1e-3

            self.RN50_INAT = True
            self.NTSNET = False

            # Determine if you want to use 1st, 2nd or 3rd NNs (in each class) to pair with your input to train AdvNet.
            self.negative_order = 1

            if self.set == 'test':
                self.data_dir = f'{self.parent_dir}/{self.test_path}'  # CUB test folder
            else:
                self.data_dir = f'{self.parent_dir}/datasets/CUB/advnet/{self.set}'  # CUB train folder

            self.Q = 10  # Q values for building positives and negatives
            self.faiss_npy_file = f'{self.prj_dir}/faiss/cub/INAT_{self.RN50_INAT}_top{self.Q}_k{self.k_value}_rn{self.resnet}_{self.set}-set_NN{self.negative_order}th.npy'

            self.aug_data_dir = f'{self.data_dir}_INAT_{self.RN50_INAT}_top{self.Q}_rn{self.resnet}_NN{self.negative_order}th'

            if self.VisionTransformer is True:
                self.faiss_npy_file = f'{self.prj_dir}/faiss/cub/ViT_top{self.Q}_k{self.k_value}_{self.set}-set_NN{self.negative_order}th.npy'

                self.aug_data_dir = f'{self.data_dir}_ViT_INAT_{self.RN50_INAT}_top{self.Q}_rn{self.resnet}_NN{self.negative_order}th'

            self.N = 4  # Depth of self-attention
            self.M = 4  # Depth of cross-attention
            self.L = 2  # Depth of transformer
            self.extension = '.jpg'
        else:
            raise ValueError("No valid training mode set.")

In [ ]:
# --------------------------------------
# The ResNet models to be used for this assignment are ResNet-18 and ResNet-34
# --------------------------------------

running_params = RunningParams('CUB', version = 18, seed = 42)

# --------------------------------------
# For the version option, we can include ResNet-18, ResNet-34, or even ResNet-50.
# --------------------------------------

if running_params.resnet == 50:
    model = torchvision.models.resnet50(pretrained=False).to(device)
elif running_params.resnet == 34:
    model = torchvision.models.resnet34(pretrained=False).to(device)
elif running_params.resnet == 18:
    model = torchvision.models.resnet18(pretrained=False).to(device)
    
for param in model.parameters():
    param.requires_grad = True

We first evaluate the performance using ResNet-18.

In [ ]:
for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25], gamma=0.1)

def test_model(model):
    model.eval()
    print('---------- Start Model Evaluations -----------')
    running_loss = 0.0
    running_corrects = 0

    for _, (data, target) in enumerate(tqdm(cub_val_loader, desc="Evaluating")):
        data = data.to(device)
        target = target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * target.size(0)
        running_corrects += torch.sum(preds == target.data)

    epoch_loss = running_loss / len(cub_val_dataset.labels)
    epoch_acc = running_corrects.double() / len(cub_val_dataset.labels)

    print("-" * 10)
    print("loss: {:.4f}, acc: {:.4f}".format(epoch_loss, epoch_acc))
    print('---------- End Model Evaluations -----------')

def train_model(model, criterion, optimizer, num_epochs=3):

    train_len = len(cub_train_dataset.labels)
    val_len = len(cub_val_dataset.labels)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    best_model_wts = model.state_dict()
    best_val_acc = 0.0
    best_model_path = f'ResNet_{version_num}best_model.pth'

    start = time.time()
    for epoch in range(num_epochs):

        if epoch != 0 :
            print('---------------------------------------------')
            
        print(f"Starting epoch : {epoch + 1}/{num_epochs}")

        running_loss = 0.0
        running_corrects = 0
        val_running_loss = 0.0
        val_running_corrects = 0

        # Update model
        model.train()
        for _, (data, target) in enumerate(tqdm(cub_train_loader, desc=f"Training Epoch {epoch + 1}")):
            data = data.to(device)
            target = target.to(device)
            outputs = model(data)

            loss = criterion(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * target.size(0)
            running_corrects += torch.sum(preds == target.data)

        # Calculate validation score
        model.eval()
        for _, (data, target) in enumerate(tqdm(cub_val_loader, desc="Validating")):
            data = data.to(device)
            target = target.to(device)
            outputs = model(data)

            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, target)
            val_running_loss += loss.item() * target.size(0)
            val_running_corrects += torch.sum(preds == target.data)

        epoch_loss = running_loss / train_len
        epoch_acc = running_corrects.double() / train_len
        
        # Note that step should be called after 25 epochs
        scheduler.step(epoch_loss)
        
        if scheduler.last_epoch in scheduler.milestones :
            print(f"Scheduler step executed: learning rate updated at epoch {scheduler.last_epoch}")
        
        val_epoch_loss = val_running_loss / val_len
        val_epoch_acc = val_running_corrects.double() / val_len

        train_losses.append(epoch_loss)
        val_losses.append(val_epoch_loss)
        train_accuracies.append(epoch_acc.item())
        val_accuracies.append(val_epoch_acc.item())

        # Save the best model
        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            best_model_wts = model.state_dict()
            torch.save(best_model_wts, best_model_path)
            print(f"Best model updated and saved to {best_model_path} with validation accuracy: {best_val_acc:.4f}")

        end = time.time()
        between_time = end - start
        if epoch % 1 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, time = {between_time:.1f} / {between_time * num_epochs / (epoch + 1):.1f}")
            print(f"Loss: {epoch_loss:.4f}, ACC: {epoch_acc * 100:.2f}%, Val_Loss: {val_epoch_loss:.4f}, Val_Acc: {val_epoch_acc * 100:.2f}%")

    # Load the best model weights
    model.load_state_dict(best_model_wts)
    print('---------- End Training Process -----------')
    print(f"Best model saved at: {best_model_path}")

    # Plot losses
    plt.figure()
    plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', color='darkgreen')
    plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', color='navy')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Plot')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    # Plot accuracies
    plt.figure()
    plt.plot(range(1, num_epochs + 1), train_accuracies, label='Train Accuracy', color='darkgreen')
    plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='navy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Plot')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    return model

print('---------- Starting Training Process -----------')
model_trained = train_model(model, criterion, optimizer, num_epochs=50)
print('---------- Training Complete -----------')

test_model(model_trained)
print('---------- Evaluation Complete -----------')


From the above results, we can infer that overfitting has occurred.

We will employ the following methodologies:

1. Pre-training and (full) fine-tuning
2. Data Augmentation
3. Ensemble

In [ ]:
# Model_save

torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),   
}, 'pre_trained_ResNet_18_original')

## 1. Pre-training and (full)fine-tuning

### Pre-training With CIFAR-10

In [ ]:
# Define Pretraining hyperparameters
pre_epochs = 40
pre_lr = 0.01
pre_optimizer = optim.Adam(model.parameters())
pre_criterion = nn.CrossEntropyLoss()

scheduler = optim.lr_scheduler.MultiStepLR(pre_optimizer, milestones=[20], gamma=0.1)
model.fc = nn.Sequential(nn.Linear(512, 10)).to(device)

In [ ]:

train_len = len(cifar10_train_dataset.targets)

train_losses = []
train_accuracies = []

start = time.time()
for epoch in range(pre_epochs):

    if epoch != 0 :
        print('---------------------------------------------')
        
    print(f"Starting epoch : {epoch + 1}/{pre_epochs}")

    running_loss = 0.0
    running_corrects = 0
    val_running_loss = 0.0
    val_running_corrects = 0

    # Update model
    model.train()
    for _, (data, target) in enumerate(tqdm(cifar10_train_loader, desc=f"Training Epoch {epoch + 1}")):
        data = data.to(device)
        target = target.to(device)
        outputs = model(data)

        loss = pre_criterion(outputs, target)
        pre_optimizer.zero_grad()
        loss.backward()
        pre_optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * target.size(0)
        running_corrects += torch.sum(preds == target.data)

    epoch_loss = running_loss / train_len
    epoch_acc = running_corrects.double() / train_len
    
    # Note that step should be called after 20 epochs
    scheduler.step(epoch_loss)
    
    if scheduler.last_epoch in scheduler.milestones :
        print(f"Scheduler step executed: learning rate updated at epoch {scheduler.last_epoch}")

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc.item())

    end = time.time()
    between_time = end - start
    if epoch % 1 == 0:
        print(f"Epoch {epoch + 1}/{pre_epochs}, time = {between_time:.1f} / {between_time * pre_epochs / (epoch + 1):.1f}")
        print(f"Loss: {epoch_loss:.4f}, ACC: {epoch_acc * 100:.2f}%")

print('---------- End Training Process -----------')

### Pre-training with CIFAR-100

In [ ]:
# Define Pretraining hyperparameters
pre_epochs = 40
pre_lr = 0.01

scheduler = optim.lr_scheduler.MultiStepLR(pre_optimizer, milestones=[20], gamma=0.1)
model.fc = nn.Sequential(nn.Linear(512, 100)).to(device)

In [ ]:
train_len = len(cifar100_train_dataset.targets)

train_losses = []
train_accuracies = []

start = time.time()
for epoch in range(pre_epochs):

    if epoch != 0 :
        print('---------------------------------------------')
        
    print(f"Starting epoch : {epoch + 1}/{pre_epochs}")

    running_loss = 0.0
    running_corrects = 0
    val_running_loss = 0.0
    val_running_corrects = 0

    # Update model
    model.train()
    for _, (data, target) in enumerate(tqdm(cifar100_train_loader, desc=f"Training Epoch {epoch + 1}")):
        data = data.to(device)
        target = target.to(device)
        outputs = model(data)

        loss = pre_criterion(outputs, target)
        pre_optimizer.zero_grad()
        loss.backward()
        pre_optimizer.step()

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * target.size(0)
        running_corrects += torch.sum(preds == target.data)

    epoch_loss = running_loss / train_len
    epoch_acc = running_corrects.double() / train_len

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc.item())

    end = time.time()
    between_time = end - start
    if epoch % 1 == 0:
        print(f"Epoch {epoch + 1}/{pre_epochs}, time = {between_time:.1f} / {between_time * pre_epochs / (epoch + 1):.1f}")
        print(f"Loss: {epoch_loss:.4f}, ACC: {epoch_acc * 100:.2f}%")

print('---------- End Training Process -----------')

### ModelSave.

In [ ]:
# Model Save
# torch.save({
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': pre_optimizer.state_dict(),
#     'epoch': epoch, 
#     'loss': loss      
# }, 'pre_trained_ResNet_18')

# Load the our model
# pre_train_model = torch.load("pre_trained_ResNet_18")
# pretrained_dict = pre_train_model['model_state_dict']

# modified_dict = {k.replace("fc.0", "fc"): v for k, v in pretrained_dict.items()}

# model = torchvision.models.resnet18(pretrained=False).to(device)
# model.fc = nn.Linear(512, 200)
# model.load_state_dict(modified_dict, strict=False)

Model saving causes issues with kernel crashes, so the pre-trained model is saved instead.

### Fine-tuning only head part.

In [ ]:
model.fc = nn.Sequential(nn.Linear(512, 200)).to(device)

In [ ]:
version_num = 18
running_params = RunningParams('CUB', version = version_num, seed = 42)

for name, param in model.named_parameters():
    if "fc" in name:  
        param.requires_grad = True
    else:
        param.requires_grad = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

def test_model(model):
    model.eval()
    print('---------- Start Model Evaluations -----------')
    running_loss = 0.0
    running_corrects = 0

    for _, (data, target) in enumerate(tqdm(cub_val_loader, desc="Evaluating")):
        data = data.to(device)
        target = target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * target.size(0)
        running_corrects += torch.sum(preds == target.data)

    epoch_loss = running_loss / len(cub_val_dataset.labels)
    epoch_acc = running_corrects.double() / len(cub_val_dataset.labels)

    print("-" * 10)
    print("loss: {:.4f}, acc: {:.4f}".format(epoch_loss, epoch_acc))
    print('---------- End Model Evaluations -----------')

def train_model(model, criterion, optimizer, num_epochs=3):

    train_len = len(cub_train_dataset.labels)
    val_len = len(cub_val_dataset.labels)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    best_model_wts = model.state_dict()
    best_val_acc = 0.0
    best_model_path = f'ResNet_{version_num}best_model.pth'

    start = time.time()
    for epoch in range(num_epochs):

        if epoch != 0 :
            print('---------------------------------------------')
            
        print(f"Starting epoch : {epoch + 1}/{num_epochs}")

        running_loss = 0.0
        running_corrects = 0
        val_running_loss = 0.0
        val_running_corrects = 0

        # Update model
        model.train()
        for _, (data, target) in enumerate(tqdm(cub_train_loader, desc=f"Training Epoch {epoch + 1}")):
            data = data.to(device)
            target = target.to(device)
            outputs = model(data)

            loss = criterion(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * target.size(0)
            running_corrects += torch.sum(preds == target.data)

        # Calculate validation score
        model.eval()
        for _, (data, target) in enumerate(tqdm(cub_val_loader, desc="Validating")):
            data = data.to(device)
            target = target.to(device)
            outputs = model(data)

            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, target)
            val_running_loss += loss.item() * target.size(0)
            val_running_corrects += torch.sum(preds == target.data)

        epoch_loss = running_loss / train_len
        epoch_acc = running_corrects.double() / train_len
        val_epoch_loss = val_running_loss / val_len
        val_epoch_acc = val_running_corrects.double() / val_len

        train_losses.append(epoch_loss)
        val_losses.append(val_epoch_loss)
        train_accuracies.append(epoch_acc.item())
        val_accuracies.append(val_epoch_acc.item())

        # Save the best model
        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            best_model_wts = model.state_dict()
            torch.save(best_model_wts, best_model_path)
            print(f"Best model updated and saved to {best_model_path} with validation accuracy: {best_val_acc:.4f}")

        end = time.time()
        between_time = end - start
        if epoch % 1 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, time = {between_time:.1f} / {between_time * num_epochs / (epoch + 1):.1f}")
            print(f"Loss: {epoch_loss:.4f}, ACC: {epoch_acc * 100:.2f}%, Val_Loss: {val_epoch_loss:.4f}, Val_Acc: {val_epoch_acc * 100:.2f}%")

    # Load the best model weights
    model.load_state_dict(best_model_wts)
    print('---------- End Training Process -----------')
    print(f"Best model saved at: {best_model_path}")

    # Plot losses
    plt.figure()
    plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', color='darkgreen')
    plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', color='navy')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Plot')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    # Plot accuracies
    plt.figure()
    plt.plot(range(1, num_epochs + 1), train_accuracies, label='Train Accuracy', color='darkgreen')
    plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='navy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Plot')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    return model

print('---------- Starting Training Process -----------')
model_trained = train_model(model, criterion, optimizer, num_epochs=100)
print('---------- Training Complete -----------')

test_model(model_trained)
print('---------- Evaluation Complete -----------')


From the above results, We can say that we need to use full-finetuning.

In [ ]:
model.fc = nn.Sequential(nn.Linear(512, 200)).to(device)

version_num = 18
running_params = RunningParams('CUB', version = version_num, seed = 42)

for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

def test_model(model):
    model.eval()
    print('---------- Start Model Evaluations -----------')
    running_loss = 0.0
    running_corrects = 0

    for _, (data, target) in enumerate(tqdm(cub_val_loader, desc="Evaluating")):
        data = data.to(device)
        target = target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * target.size(0)
        running_corrects += torch.sum(preds == target.data)

    epoch_loss = running_loss / len(cub_val_dataset.labels)
    epoch_acc = running_corrects.double() / len(cub_val_dataset.labels)

    print("-" * 10)
    print("loss: {:.4f}, acc: {:.4f}".format(epoch_loss, epoch_acc))
    print('---------- End Model Evaluations -----------')

def train_model(model, criterion, optimizer, num_epochs=3):

    train_len = len(cub_train_dataset.labels)
    val_len = len(cub_val_dataset.labels)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    best_model_wts = model.state_dict()
    best_val_acc = 0.0
    best_model_path = f'ResNet_{version_num}best_model.pth'

    start = time.time()
    for epoch in range(num_epochs):

        if epoch != 0 :
            print('---------------------------------------------')
            
        print(f"Starting epoch : {epoch + 1}/{num_epochs}")

        running_loss = 0.0
        running_corrects = 0
        val_running_loss = 0.0
        val_running_corrects = 0

        # Update model
        model.train()
        for _, (data, target) in enumerate(tqdm(cub_train_loader, desc=f"Training Epoch {epoch + 1}")):
            data = data.to(device)
            target = target.to(device)
            outputs = model(data)

            loss = criterion(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * target.size(0)
            running_corrects += torch.sum(preds == target.data)

        # Calculate validation score
        model.eval()
        for _, (data, target) in enumerate(tqdm(cub_val_loader, desc="Validating")):
            data = data.to(device)
            target = target.to(device)
            outputs = model(data)

            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, target)
            val_running_loss += loss.item() * target.size(0)
            val_running_corrects += torch.sum(preds == target.data)

        epoch_loss = running_loss / train_len
        epoch_acc = running_corrects.double() / train_len
        val_epoch_loss = val_running_loss / val_len
        val_epoch_acc = val_running_corrects.double() / val_len

        train_losses.append(epoch_loss)
        val_losses.append(val_epoch_loss)
        train_accuracies.append(epoch_acc.item())
        val_accuracies.append(val_epoch_acc.item())

        # Save the best model
        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            best_model_wts = model.state_dict()
            torch.save(best_model_wts, best_model_path)
            print(f"Best model updated and saved to {best_model_path} with validation accuracy: {best_val_acc:.4f}")

        end = time.time()
        between_time = end - start
        if epoch % 1 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, time = {between_time:.1f} / {between_time * num_epochs / (epoch + 1):.1f}")
            print(f"Loss: {epoch_loss:.4f}, ACC: {epoch_acc * 100:.2f}%, Val_Loss: {val_epoch_loss:.4f}, Val_Acc: {val_epoch_acc * 100:.2f}%")

    # Load the best model weights
    model.load_state_dict(best_model_wts)
    print('---------- End Training Process -----------')
    print(f"Best model saved at: {best_model_path}")

    # Plot losses
    plt.figure()
    plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', color='darkgreen')
    plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', color='navy')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Plot')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    # Plot accuracies
    plt.figure()
    plt.plot(range(1, num_epochs + 1), train_accuracies, label='Train Accuracy', color='darkgreen')
    plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='navy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Plot')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    return model

print('---------- Starting Training Process -----------')
model_trained = train_model(model, criterion, optimizer, num_epochs=100)
print('---------- Training Complete -----------')

test_model(model_trained)
print('---------- Evaluation Complete -----------')

However, it can be observed that the performance is still unsatisfactory even with the full fine-tuning method.

# 2. Data Augmentation

In [ ]:
class CUB_Dataset(Dataset):
    def __init__(self, img_file, label_file, batch_size, transform=None):
        self.img = np.load(img_file)         # shape: (N, H, W, C), dtype=float32 가정
        self.labels = np.load(label_file)    # shape: (N,)
        self.transform = transform
        self.batch_size = batch_size

    def __len__(self):
        return len(self.img)

    def __getitem__(self, idx):
        image = self.img[idx]   
        label = self.labels[idx]
        
        image = image.astype(np.uint8)

        if self.transform:
            image = self.transform(image)

        return image, label

# --------------------------------------------------
# Augmentation Transformations (Training)
# --------------------------------------------------
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=0.5),  
    # transforms.RandomRotation(degrees=5),   
    # transforms.ColorJitter(brightness=0.1,  
    #                        contrast=0.1,
    #                        saturation=0.1,
    #                        hue=0.05),
    # transforms.RandomResizedCrop(size=(224, 224), scale=(0.9, 1.0)),  
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# --------------------------------------------------
# Validation Transform (No Augmentation)
# --------------------------------------------------
val_transform = transforms.Compose([
    transforms.ToPILImage(),                    # numpy(uint8) -> PIL
    # transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])

# --------------------------------------------------
# Dataset & DataLoader
# --------------------------------------------------

# Training dataset (with augmentation)
cub_train_dataset = CUB_Dataset(
    img_file="./CUB_train_images.npy",
    label_file="./CUB_train_labels.npy",
    batch_size=64,
    transform=train_transform
)
cub_train_loader = torch.utils.data.DataLoader(
    cub_train_dataset,
    batch_size=64,
    shuffle=True
)

# Validation dataset (minimal transforms)
cub_val_dataset = CUB_Dataset(
    img_file="./CUB_val_images.npy",
    label_file="./CUB_val_labels.npy",
    batch_size=64,
    transform=val_transform
)
cub_val_loader = torch.utils.data.DataLoader(
    cub_val_dataset,
    batch_size=64,
    shuffle=False
)


First, we evaluate the model by training only the head of the model using the CUB-200 dataset.

In [ ]:
model.fc = nn.Sequential(nn.Linear(512, 200)).to(device)
version_num = 18
running_params = RunningParams('CUB', version = version_num, seed = 42)

for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

def test_model(model):
    model.eval()
    print('---------- Start Model Evaluations -----------')
    running_loss = 0.0
    running_corrects = 0

    for _, (data, target) in enumerate(tqdm(cub_val_loader, desc="Evaluating")):
        data = data.to(device)
        target = target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * target.size(0)
        running_corrects += torch.sum(preds == target.data)

    epoch_loss = running_loss / len(cub_val_dataset.labels)
    epoch_acc = running_corrects.double() / len(cub_val_dataset.labels)

    print("-" * 10)
    print("loss: {:.4f}, acc: {:.4f}".format(epoch_loss, epoch_acc))
    print('---------- End Model Evaluations -----------')

def train_model(model, criterion, optimizer, num_epochs=3):

    train_len = len(cub_train_dataset.labels)
    val_len = len(cub_val_dataset.labels)

    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    best_model_wts = model.state_dict()
    best_val_acc = 0.0
    best_model_path = f'ResNet_{version_num}best_model.pth'

    start = time.time()
    for epoch in range(num_epochs):

        if epoch != 0 :
            print('---------------------------------------------')
            
        print(f"Starting epoch : {epoch + 1}/{num_epochs}")

        running_loss = 0.0
        running_corrects = 0
        val_running_loss = 0.0
        val_running_corrects = 0

        # Update model
        model.train()
        for _, (data, target) in enumerate(tqdm(cub_train_loader, desc=f"Training Epoch {epoch + 1}")):
            data = data.to(device)
            target = target.to(device)
            outputs = model(data)

            loss = criterion(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * target.size(0)
            running_corrects += torch.sum(preds == target.data)

        # Calculate validation score
        model.eval()
        for _, (data, target) in enumerate(tqdm(cub_val_loader, desc="Validating")):
            data = data.to(device)
            target = target.to(device)
            outputs = model(data)

            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, target)
            val_running_loss += loss.item() * target.size(0)
            val_running_corrects += torch.sum(preds == target.data)

        epoch_loss = running_loss / train_len
        epoch_acc = running_corrects.double() / train_len
        val_epoch_loss = val_running_loss / val_len
        val_epoch_acc = val_running_corrects.double() / val_len

        train_losses.append(epoch_loss)
        val_losses.append(val_epoch_loss)
        train_accuracies.append(epoch_acc.item())
        val_accuracies.append(val_epoch_acc.item())

        # Save the best model
        if val_epoch_acc > best_val_acc:
            best_val_acc = val_epoch_acc
            best_model_wts = model.state_dict()
            torch.save(best_model_wts, best_model_path)
            print(f"Best model updated and saved to {best_model_path} with validation accuracy: {best_val_acc:.4f}")

        end = time.time()
        between_time = end - start
        if epoch % 1 == 0:
            print(f"Epoch {epoch + 1}/{num_epochs}, time = {between_time:.1f} / {between_time * num_epochs / (epoch + 1):.1f}")
            print(f"Loss: {epoch_loss:.4f}, ACC: {epoch_acc * 100:.2f}%, Val_Loss: {val_epoch_loss:.4f}, Val_Acc: {val_epoch_acc * 100:.2f}%")

    # Load the best model weights
    model.load_state_dict(best_model_wts)
    print('---------- End Training Process -----------')
    print(f"Best model saved at: {best_model_path}")

    # Plot losses
    plt.figure()
    plt.plot(range(1, num_epochs + 1), train_losses, label='Train Loss', color='darkgreen')
    plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', color='navy')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Plot')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    # Plot accuracies
    plt.figure()
    plt.plot(range(1, num_epochs + 1), train_accuracies, label='Train Accuracy', color='darkgreen')
    plt.plot(range(1, num_epochs + 1), val_accuracies, label='Validation Accuracy', color='navy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Plot')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    return model

print('---------- Starting Training Process -----------')
model_trained = train_model(model, criterion, optimizer, num_epochs=100)
print('---------- Training Complete -----------')

test_model(model_trained)
print('---------- Evaluation Complete -----------')


# 3. Ensemble

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_file, transform=None):
        self.img = np.load(img_file)
        self.transform = transform

    def __len__(self):
        return len(self.img)

    def __getitem__(self, idx):
        image = self.img[idx]
        if self.transform is not None:
            image = self.transform(image)
        return image

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_dataset = TestDataset(img_file="./CUB_test_images.npy", transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

## ResNet 18 Ensemble

In [ ]:
import time
import os
os.environ["MKL_NUM_THREADS"] = "4" 
os.environ["NUMEXPR_NUM_THREADS"] = "1"  
os.environ["OMP_NUM_THREADS"] = "1" 
import shutil
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

from copy import deepcopy
from datetime import datetime

import logging
import argparse

criterion = nn.CrossEntropyLoss()

def inference_no_label_ensemble(model_list, test_loader, save_path="./Test_ensemble_results.npy"):

    for m in model_list:
        m.eval()

    all_preds = []
    with torch.inference_mode():
        for images in tqdm(test_loader, desc="Ensemble Inference(no-label)"):
            images = images.to(device)
            if images.dim() == 3:
                images = images.unsqueeze(1)
            if images.shape[1] == 1:
                images = images.repeat(1, 3, 1, 1)

            prob_list = []
            for m in model_list:
                outputs = m(images)
                probs = F.softmax(outputs, dim=1)
                prob_list.append(probs)

            avg_prob = torch.mean(torch.stack(prob_list, dim=0), dim=0) 
            _, preds = torch.max(avg_prob, 1)
            all_preds.append(preds.cpu())

    all_preds = torch.cat(all_preds, dim=0).numpy()
    np.save(save_path, all_preds)
    print(f"[Ensemble Inference] Test predictions saved to {save_path} (shape={all_preds.shape})")
    
def ensemble_inference_val(model_list, val_loader, device):

    for m in model_list:
        m.eval()

    running_corrects = 0
    total = 0

    with torch.inference_mode():
        for images, labels in tqdm(val_loader, desc="Ensemble Inference(val)"):
            images, labels = images.to(device), labels.to(device)
            if images.dim() == 3:
                images = images.unsqueeze(1) 
            if images.shape[1] == 1:
                images = images.repeat(1, 3, 1, 1) 
            prob_list = []
            for m in model_list:
                outputs = m(images)           
                probs = F.softmax(outputs, dim=1)
                prob_list.append(probs)

            avg_prob = torch.mean(torch.stack(prob_list, dim=0), dim=0) 
            _, preds = torch.max(avg_prob, 1)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

    val_acc = (running_corrects.double() / total).item() if total > 0 else 0.0
    return val_acc


class Reg_ho_ens(object):
    def __init__(self, args, LOG, **kwargs):
        self.args = args
        self.LOG = LOG
        self.measures_name = ['CrossEntropy', 'acc']
        
        for key, value in args.__dict__.items():
            setattr(self, key, value)
        for key, value in kwargs.items():
            setattr(self, key, value)
        
        self.train_loader = None
        self.valid_loader = None
        self.test_loader = None
        self.model_list = []
        self.optimizer_list = []
        self.scheduler_list = []
        
        self.best_state_dict_total = {}

    def _fix_seed(self):
        random.seed(self.seed)
        np.random.seed(self.seed)
        torch.manual_seed(self.seed)
        torch.cuda.manual_seed(self.seed)
        torch.backends.cudnn.deterministic = True

    def _create_model(self, version=18):
        
        if version == 18:
            model = torchvision.models.resnet18(pretrained=False).to(device)
            model.fc = nn.Sequential(nn.Linear(512, 200)).to(device)
            model = model.to(device)
        else:
            print("You can only use ResNet-18 in this code.")
            model = None
        return model

    def _make_loaders(self):
        """
        train_loader: cub_train_loader (data, label)
        valid_loader: cub_val_loader   (data, label)
        test_loader : test_dataset     (data only, no label)
        """
        self.train_loader = cub_train_loader
        self.valid_loader = cub_val_loader
        self.test_loader  = test_loader

    def _define_model_and_optimizer(self, optim='adam'):
        self.model_list = [self._create_model(version=18) for _ in range(self.n_ens)]
        if optim == 'adam':
            self.optimizer_list = [
                torch.optim.Adam(model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
                for model in self.model_list
            ]
        else:
            self.optimizer_list = [
                torch.optim.SGD(model.parameters(), lr=self.lr, momentum=0.9, weight_decay=self.weight_decay)
                for model in self.model_list
            ]
        self.scheduler_list = [
            torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.lr_schedule, gamma=0.1)
            for optimizer in self.optimizer_list
        ]

    def _train_one_epoch(self, model, optimizer, epoch_idx, model_idx):

        model.train()
        running_loss = 0.0
        pbar = tqdm(self.train_loader, desc=f"[Train] Model{model_idx + 1} Epoch{epoch_idx + 1}")
        for batch_data in pbar:
            inputs, targets = batch_data
            inputs, targets = inputs.to(device), targets.to(device)

            if inputs.dim() == 3:
                inputs = inputs.unsqueeze(1)   
            if inputs.shape[1] == 1:
                inputs = inputs.repeat(1,3,1,1)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_loss = running_loss / len(self.train_loader)
        return epoch_loss

    def _eval_accuracy(self, model, loader, desc=""):

        model.eval()
        running_corrects = 0
        total = 0
        with torch.no_grad():
            pbar = tqdm(loader, desc=desc)
            for batch_data in pbar:
                if len(batch_data) == 2:
                    data, target = batch_data
                    data, target = data.to(device), target.to(device)
                    if data.dim() == 3:
                        data = data.unsqueeze(1) 
                    if data.shape[1] == 1:
                        data = data.repeat(1,3,1,1)
                    
                    outputs = model(data)
                    _, preds = torch.max(outputs, 1)
                    running_corrects += torch.sum(preds == target.data)
                    total += target.size(0)
                else:

                    pass
        if total == 0:
            return 0.0
        acc = (running_corrects.double() / total).item()
        return acc
    
    def _train_ens(self):

        self.train_losses = {}
        self.val_accs = {}
        self.test_single_acc = {}

        for num, (model, optimizer, scheduler) in enumerate(zip(self.model_list, self.optimizer_list, self.scheduler_list)):
            print(f"====== Starting training for Model {num + 1} / n_ens={self.n_ens} ======")
            train_loss_list = []
            val_acc_list = []

            for epoch in range(self.epochs):
                epoch_loss = self._train_one_epoch(model, optimizer, epoch, num)

                # train acc
                train_acc = self._eval_accuracy(model, self.train_loader, desc=f"[Train-ACC] Model{num + 1} Epoch{epoch + 1}")
                # val acc
                val_acc   = self._eval_accuracy(model, self.valid_loader, desc=f"[Val-ACC] Model{num + 1} Epoch{epoch + 1}")
                
                train_loss_list.append(epoch_loss)
                val_acc_list.append(val_acc)

                print(f"[Model : {num + 1}, Epoch : {epoch+1}/{self.epochs}] TrainLoss={epoch_loss:.4f} TrainAcc={train_acc*100:.2f}% ValAcc={val_acc*100:.2f}%")
                print('-----------------------------')
                
                scheduler.step()

            single_test_acc = self._eval_accuracy(model, self.test_loader)

            self.train_losses[num] = train_loss_list
            self.val_accs[num] = val_acc_list
            self.test_single_acc[num] = single_test_acc

    def plot_curve(self, save_dir):
        """
        여기서는 train_losses와 val_accs를 각각 그려볼 수 있음.
        """
        # Train Loss curve
        title = 'loss curve of train'
        dpi = 80
        width, height = 1200, 800
        figsize = width / float(dpi), height / float(dpi)

        fig = plt.figure(figsize=figsize)
        x_axis = np.array([i for i in range(self.epochs)])
        plt.xlim(0, self.epochs)

        if len(self.train_losses) > 0:
            y_max = max([max(self.train_losses[k]) for k in self.train_losses])
            plt.ylim(0, y_max*1.1)
        plt.grid()
        plt.title(title, fontsize=20)
        plt.xlabel('Epoch', fontsize=16)
        plt.ylabel('Train Loss', fontsize=16)

        for e, losses in self.train_losses.items():
            plt.plot(x_axis, losses, linestyle='-', label=f'Model-{e}', lw=2)
        fig.savefig(os.path.join(save_dir, "train_losses_plot.png"), dpi=dpi, bbox_inches='tight')
        plt.close(fig)

        # Val ACC curve
        title = 'Accuracy curve of validation'
        fig = plt.figure(figsize=figsize)
        x_axis = np.array([i for i in range(self.epochs)])
        plt.xlim(0, self.epochs)
        if len(self.val_accs) > 0:
            y_max = max([max(self.val_accs[k]) for k in self.val_accs])
            plt.ylim(0, y_max*1.1)
        plt.grid()
        plt.title(title, fontsize=20)
        plt.xlabel('Epoch', fontsize=16)
        plt.ylabel('Val ACC', fontsize=16)

        for e, accs in self.val_accs.items():
            plt.plot(x_axis, accs, linestyle='-', label=f'Model-{e}-valACC', lw=2)
        fig.savefig(os.path.join(save_dir, "val_acc_plot.png"), dpi=dpi, bbox_inches='tight')
        plt.close(fig)

    def _save_checkpoint(self, state, name):
        filename = 'checkpoint_{}.ckpt'.format(name)
        checkpoint_path = os.path.join(self.save_dir, filename)
        torch.save(state, checkpoint_path)


def main(args):
    main_category = 'teachers_best'
    code_specified_dir = os.path.join(args.save_dir, main_category, args.dataset)
    save_dir = os.path.join(code_specified_dir, '_'.join([args.arch]+ [str(x) for x in args.h_vec]))
    args.save_dir = save_dir
    shutil.rmtree(save_dir, ignore_errors=True)
    os.makedirs(save_dir, exist_ok=True)

    LOG = logging.getLogger('main')
    LOG.setLevel(logging.INFO)
    log_file = os.path.join(args.save_dir, 'log.txt')
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.INFO)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.WARNING)
    LOG.addHandler(stream_handler)
    LOG.addHandler(file_handler)

    if torch.cuda.is_available():
        torch.cuda.set_device(args.gpu_number)

    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = True

    reg = Reg_ho_ens(args, LOG)

    reg.seed = args.data_seed
    reg._fix_seed()
    reg._make_loaders()
    reg._define_model_and_optimizer(optim=args.sgd_methods)
    reg._train_ens()
    reg.plot_curve(save_dir)

    LOG.info("Done training.")
    print(save_dir)


    for idx, model in enumerate(reg.model_list):
        best_ckpt_path = os.path.join(save_dir, f"checkpoint_{idx}_best_acc.ckpt")
        if os.path.exists(best_ckpt_path):
            best_ckpt = torch.load(best_ckpt_path)
            model.load_state_dict(best_ckpt['state_dict'])
            print(f"Loaded best_acc checkpoint for model {idx} from {best_ckpt_path}")
        else:
            print(f"[Warning] Best checkpoint not found for model {idx}, using last epoch param.")
            
    # ----- 앙상블 Validation Accuracy 계산 -----
    val_acc = ensemble_inference_val(reg.model_list, reg.valid_loader, device)
    print(f"\n[Ensemble] Validation Accuracy = {val_acc*100:.2f}%")

    ensemble_output_path = os.path.join(save_dir, "ensemble_predictions_bestAcc.npy")
    inference_no_label_ensemble(reg.model_list, reg.test_loader, save_path=ensemble_output_path)
    print(f"Best checkpoint ensemble inference saved to {ensemble_output_path}")


if __name__ == '__main__':
    
    def create_parser():
        parser = argparse.ArgumentParser(description='KD : teacher ensembles')
        parser.add_argument('--explanation', default='', type=str)
        parser.add_argument('--save_dir', default='.', type=str)
        parser.add_argument('--data_dir', default='.', type=str)
        parser.add_argument('--seed', default=1, type=int)
        parser.add_argument('--data_seed', default=1, type=int)
        parser.add_argument('--gpu_number', default=0, type=int)
        parser.add_argument('--workers', default=0, type=int)
        parser.add_argument('--batch_size', default=100, type=int)
        parser.add_argument('--arch', default='ResNet18', type=str)
        parser.add_argument('--activation', default='ReLU', type=str)
        parser.add_argument('--sgd_methods', default='adam', type=str)
        parser.add_argument('--n_ens', default=10, type=int)
        parser.add_argument('--h_vec', nargs='*', type=int, default=[50])
        parser.add_argument('--dataset', default='CUB', type=str)
        parser.add_argument('--lr', default=1e-3, type=float)
        parser.add_argument('--lr_schedule', nargs='*', type=int, default=[600])
        parser.add_argument('--weight_decay', default=1e-4, type=float)
        parser.add_argument('--epochs', default=50, type=int)
        parser.add_argument('--evaluation_epochs', default=1, type=int)
        parser.add_argument('--ratio_valid', default=0.0, type=float)
        parser.add_argument('--print_freq', default=100, type=int)
        return parser

    for dataset in ['CUB']:
        args = create_parser().parse_args([])
        args.dataset = dataset
        args.lr = 1e-3
        args.epochs = 50
        args.n_ens = 10
        main(args)

[Val-ACC] Model6 Epoch35: 100%|██████████| 46/46 [00:01<00:00, 24.73it/s]


[Model : 6, Epoch : 35/50] TrainLoss=2.9310 TrainAcc=29.28% ValAcc=14.88%
-----------------------------


[Val-ACC] Model6 Epoch36: 100%|██████████| 46/46 [00:01<00:00, 25.57it/s]


[Model : 6, Epoch : 36/50] TrainLoss=1.8020 TrainAcc=42.01% ValAcc=13.39%
-----------------------------


[Val-ACC] Model6 Epoch37: 100%|██████████| 46/46 [00:01<00:00, 24.85it/s]


...

./teachers/CUB/ResNet18_50


Ensemble Inference(val): 100%|██████████| 46/46 [00:12<00:00,  3.83it/s]



[Ensemble] Validation Accuracy = 36.76%


Ensemble Inference(no-label): 100%|██████████| 12/12 [00:12<00:00,  1.08s/it]

[Ensemble Inference] Test predictions saved to ./teachers/CUB/ResNet18_50/ensemble_predictions.npy (shape=(2897,))

[Ensemble] Test predictions saved to ./teachers/CUB/ResNet18_50/ensemble_predictions.npy


## ResNet 34 Ensemble

In [ ]:
import time
import os
os.environ["MKL_NUM_THREADS"] = "4" 
os.environ["NUMEXPR_NUM_THREADS"] = "1"  
os.environ["OMP_NUM_THREADS"] = "1" 
import shutil
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

from copy import deepcopy
from datetime import datetime

import logging
import argparse

criterion = nn.CrossEntropyLoss()

def inference_no_label_ensemble(model_list, test_loader, save_path="./Test_ensemble_results.npy"):

    for m in model_list:
        m.eval()

    all_preds = []
    with torch.inference_mode():
        for images in tqdm(test_loader, desc="Ensemble Inference(no-label)"):
            images = images.to(device)
            if images.dim() == 3:
                images = images.unsqueeze(1)
            if images.shape[1] == 1:
                images = images.repeat(1, 3, 1, 1)

            prob_list = []
            for m in model_list:
                outputs = m(images)
                probs = F.softmax(outputs, dim=1)
                prob_list.append(probs)

            avg_prob = torch.mean(torch.stack(prob_list, dim=0), dim=0) 
            _, preds = torch.max(avg_prob, 1)
            all_preds.append(preds.cpu())

    all_preds = torch.cat(all_preds, dim=0).numpy()
    np.save(save_path, all_preds)
    print(f"[Ensemble Inference] Test predictions saved to {save_path} (shape={all_preds.shape})")
    
def ensemble_inference_val(model_list, val_loader, device):

    for m in model_list:
        m.eval()

    running_corrects = 0
    total = 0

    with torch.inference_mode():
        for images, labels in tqdm(val_loader, desc="Ensemble Inference(val)"):
            images, labels = images.to(device), labels.to(device)
            if images.dim() == 3:
                images = images.unsqueeze(1) 
            if images.shape[1] == 1:
                images = images.repeat(1, 3, 1, 1) 
            prob_list = []
            for m in model_list:
                outputs = m(images)           
                probs = F.softmax(outputs, dim=1)
                prob_list.append(probs)

            avg_prob = torch.mean(torch.stack(prob_list, dim=0), dim=0) 
            _, preds = torch.max(avg_prob, 1)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

    val_acc = (running_corrects.double() / total).item() if total > 0 else 0.0
    return val_acc


class Reg_ho_ens(object):
    def __init__(self, args, LOG, **kwargs):
        self.args = args
        self.LOG = LOG
        self.measures_name = ['CrossEntropy', 'acc']
        
        for key, value in args.__dict__.items():
            setattr(self, key, value)
        for key, value in kwargs.items():
            setattr(self, key, value)
        
        self.train_loader = None
        self.valid_loader = None
        self.test_loader = None
        self.model_list = []
        self.optimizer_list = []
        self.scheduler_list = []
        
        self.best_state_dict_total = {}

    def _fix_seed(self):
        random.seed(self.seed)
        np.random.seed(self.seed)
        torch.manual_seed(self.seed)
        torch.cuda.manual_seed(self.seed)
        torch.backends.cudnn.deterministic = True

    def _create_model(self, version=34):
        
        if version == 34:
            model = torchvision.models.resnet34(pretrained=False).to(device)
            model.fc = nn.Sequential(nn.Linear(512, 200)).to(device)
            model = model.to(device)
        else:
            print("You can only use ResNet-34 in this code.")
            model = None
        return model

    def _make_loaders(self):
        """
        train_loader: cub_train_loader (data, label)
        valid_loader: cub_val_loader   (data, label)
        test_loader : test_dataset     (data only, no label)
        """
        self.train_loader = cub_train_loader
        self.valid_loader = cub_val_loader
        self.test_loader  = test_loader

    def _define_model_and_optimizer(self, optim='adam'):
        self.model_list = [self._create_model(version=34) for _ in range(self.n_ens)]
        if optim == 'adam':
            self.optimizer_list = [
                torch.optim.Adam(model.parameters(), lr=self.lr, weight_decay=self.weight_decay)
                for model in self.model_list
            ]
        else:
            self.optimizer_list = [
                torch.optim.SGD(model.parameters(), lr=self.lr, momentum=0.9, weight_decay=self.weight_decay)
                for model in self.model_list
            ]
        self.scheduler_list = [
            torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.lr_schedule, gamma=0.1)
            for optimizer in self.optimizer_list
        ]

    def _train_one_epoch(self, model, optimizer, epoch_idx, model_idx):

        model.train()
        running_loss = 0.0
        pbar = tqdm(self.train_loader, desc=f"[Train] Model{model_idx + 1} Epoch{epoch_idx + 1}")
        for batch_data in pbar:
            inputs, targets = batch_data
            inputs, targets = inputs.to(device), targets.to(device)

            if inputs.dim() == 3:
                inputs = inputs.unsqueeze(1)   
            if inputs.shape[1] == 1:
                inputs = inputs.repeat(1,3,1,1)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_loss = running_loss / len(self.train_loader)
        return epoch_loss

    def _eval_accuracy(self, model, loader, desc=""):

        model.eval()
        running_corrects = 0
        total = 0
        with torch.no_grad():
            pbar = tqdm(loader, desc=desc)
            for batch_data in pbar:
                if len(batch_data) == 2:
                    data, target = batch_data
                    data, target = data.to(device), target.to(device)
                    if data.dim() == 3:
                        data = data.unsqueeze(1) 
                    if data.shape[1] == 1:
                        data = data.repeat(1,3,1,1)
                    
                    outputs = model(data)
                    _, preds = torch.max(outputs, 1)
                    running_corrects += torch.sum(preds == target.data)
                    total += target.size(0)
                else:

                    pass
        if total == 0:
            return 0.0
        acc = (running_corrects.double() / total).item()
        return acc
    
    def _train_ens(self):

        self.train_losses = {}
        self.val_accs = {}
        self.test_single_acc = {}

        for num, (model, optimizer, scheduler) in enumerate(zip(self.model_list, self.optimizer_list, self.scheduler_list)):
            print(f"====== Starting training for Model {num + 1} / n_ens={self.n_ens} ======")
            train_loss_list = []
            val_acc_list = []

            for epoch in range(self.epochs):
                epoch_loss = self._train_one_epoch(model, optimizer, epoch, num)

                # train acc
                train_acc = self._eval_accuracy(model, self.train_loader, desc=f"[Train-ACC] Model{num + 1} Epoch{epoch + 1}")
                # val acc
                val_acc   = self._eval_accuracy(model, self.valid_loader, desc=f"[Val-ACC] Model{num + 1} Epoch{epoch + 1}")
                
                train_loss_list.append(epoch_loss)
                val_acc_list.append(val_acc)

                print(f"[Model : {num + 1}, Epoch : {epoch+1}/{self.epochs}] TrainLoss={epoch_loss:.4f} TrainAcc={train_acc*100:.2f}% ValAcc={val_acc*100:.2f}%")
                print('-----------------------------')
                
                scheduler.step()

            single_test_acc = self._eval_accuracy(model, self.test_loader)

            self.train_losses[num] = train_loss_list
            self.val_accs[num] = val_acc_list
            self.test_single_acc[num] = single_test_acc

    def plot_curve(self, save_dir):
        """
        여기서는 train_losses와 val_accs를 각각 그려볼 수 있음.
        """
        # Train Loss curve
        title = 'loss curve of train'
        dpi = 80
        width, height = 1200, 800
        figsize = width / float(dpi), height / float(dpi)

        fig = plt.figure(figsize=figsize)
        x_axis = np.array([i for i in range(self.epochs)])
        plt.xlim(0, self.epochs)

        if len(self.train_losses) > 0:
            y_max = max([max(self.train_losses[k]) for k in self.train_losses])
            plt.ylim(0, y_max*1.1)
        plt.grid()
        plt.title(title, fontsize=20)
        plt.xlabel('Epoch', fontsize=16)
        plt.ylabel('Train Loss', fontsize=16)

        for e, losses in self.train_losses.items():
            plt.plot(x_axis, losses, linestyle='-', label=f'Model-{e}', lw=2)
        fig.savefig(os.path.join(save_dir, "train_losses_plot.png"), dpi=dpi, bbox_inches='tight')
        plt.close(fig)

        # Val ACC curve
        title = 'Accuracy curve of validation'
        fig = plt.figure(figsize=figsize)
        x_axis = np.array([i for i in range(self.epochs)])
        plt.xlim(0, self.epochs)
        if len(self.val_accs) > 0:
            y_max = max([max(self.val_accs[k]) for k in self.val_accs])
            plt.ylim(0, y_max*1.1)
        plt.grid()
        plt.title(title, fontsize=20)
        plt.xlabel('Epoch', fontsize=16)
        plt.ylabel('Val ACC', fontsize=16)

        for e, accs in self.val_accs.items():
            plt.plot(x_axis, accs, linestyle='-', label=f'Model-{e}-valACC', lw=2)
        fig.savefig(os.path.join(save_dir, "val_acc_plot.png"), dpi=dpi, bbox_inches='tight')
        plt.close(fig)

    def _save_checkpoint(self, state, name):
        filename = 'checkpoint_{}.ckpt'.format(name)
        checkpoint_path = os.path.join(self.save_dir, filename)
        torch.save(state, checkpoint_path)


def main(args):
    main_category = 'teachers_best'
    code_specified_dir = os.path.join(args.save_dir, main_category, args.dataset)
    save_dir = os.path.join(code_specified_dir, '_'.join([args.arch]+ [str(x) for x in args.h_vec]))
    args.save_dir = save_dir
    shutil.rmtree(save_dir, ignore_errors=True)
    os.makedirs(save_dir, exist_ok=True)

    LOG = logging.getLogger('main')
    LOG.setLevel(logging.INFO)
    log_file = os.path.join(args.save_dir, 'log.txt')
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.INFO)
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.WARNING)
    LOG.addHandler(stream_handler)
    LOG.addHandler(file_handler)

    if torch.cuda.is_available():
        torch.cuda.set_device(args.gpu_number)

    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = True

    reg = Reg_ho_ens(args, LOG)

    reg.seed = args.data_seed
    reg._fix_seed()
    reg._make_loaders()
    reg._define_model_and_optimizer(optim=args.sgd_methods)
    reg._train_ens()
    reg.plot_curve(save_dir)

    LOG.info("Done training.")
    print(save_dir)


    for idx, model in enumerate(reg.model_list):
        best_ckpt_path = os.path.join(save_dir, f"checkpoint_{idx}_best_acc.ckpt")
        if os.path.exists(best_ckpt_path):
            best_ckpt = torch.load(best_ckpt_path)
            model.load_state_dict(best_ckpt['state_dict'])
            print(f"Loaded best_acc checkpoint for model {idx} from {best_ckpt_path}")
        else:
            print(f"[Warning] Best checkpoint not found for model {idx}, using last epoch param.")
            
    # ----- 앙상블 Validation Accuracy 계산 -----
    val_acc = ensemble_inference_val(reg.model_list, reg.valid_loader, device)
    print(f"\n[Ensemble] Validation Accuracy = {val_acc*100:.2f}%")

    ensemble_output_path = os.path.join(save_dir, "ensemble_predictions_bestAcc.npy")
    inference_no_label_ensemble(reg.model_list, reg.test_loader, save_path=ensemble_output_path)
    print(f"Best checkpoint ensemble inference saved to {ensemble_output_path}")


if __name__ == '__main__':
    
    def create_parser():
        parser = argparse.ArgumentParser(description='KD : teacher ensembles')
        parser.add_argument('--explanation', default='', type=str)
        parser.add_argument('--save_dir', default='.', type=str)
        parser.add_argument('--data_dir', default='.', type=str)
        parser.add_argument('--seed', default=1, type=int)
        parser.add_argument('--data_seed', default=1, type=int)
        parser.add_argument('--gpu_number', default=0, type=int)
        parser.add_argument('--workers', default=0, type=int)
        parser.add_argument('--batch_size', default=64, type=int)
        parser.add_argument('--arch', default='ResNet18', type=str)
        parser.add_argument('--activation', default='ReLU', type=str)
        parser.add_argument('--sgd_methods', default='adam', type=str)
        parser.add_argument('--n_ens', default=10, type=int)
        parser.add_argument('--h_vec', nargs='*', type=int, default=[50])
        parser.add_argument('--dataset', default='CUB', type=str)
        parser.add_argument('--lr', default=1e-3, type=float)
        parser.add_argument('--lr_schedule', nargs='*', type=int, default=[600])
        parser.add_argument('--weight_decay', default=1e-4, type=float)
        parser.add_argument('--epochs', default=50, type=int)
        parser.add_argument('--evaluation_epochs', default=1, type=int)
        parser.add_argument('--ratio_valid', default=0.0, type=float)
        parser.add_argument('--print_freq', default=100, type=int)
        return parser

    for dataset in ['CUB']:
        args = create_parser().parse_args([])
        args.dataset = dataset
        args.lr = 1e-3
        args.epochs = 50
        args.n_ens = 10
        main(args)

# Test and Submit

You can modify your TestDataset, but you should be mindful to align it with the training dataset and its transformations.

In [29]:
class TestDataset(Dataset):
    def __init__(self, img_file, transform=None):
        self.img = np.load(img_file)
        self.transform = transform

    def __len__(self):
        return len(self.img)

    def __getitem__(self, idx):
        image = self.img[idx]
        if self.transform is not None:
            image = self.transform(image)
        return image

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_dataset = TestDataset(img_file="./CUB_test_images.npy", transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model.eval()
test_predictions = []
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs.data, 1)
        test_predictions.extend(preds.cpu().numpy())
        
# #### The ''test_predictions'' list stores the predicted class indices for each test image.

In [ ]:
test_predictions = np.load('./teachers/CUB/ResNet18_50/ensemble_predictions.npy')
test_predictions

# --------------------------------------------------
# If you are to load the test_predictions here, you will need to use 202422105_이성은_HW3.npy.
# --------------------------------------------------

array([198,  48,  19, ..., 104, 181, 124])

## **Do not modify the cell below!!!!**


In [ ]:
def test(model, test_loader):
  model.eval()
  test_predictions = []

  with torch.inference_mode():
      for i, data in enumerate(tqdm(test_loader)):
          data = data.float().to(device)
          output = model(data)
          test_predictions.append(output.cpu())

  return torch.cat(test_predictions, dim=0)

In [ ]:
# Save test output npy file
predictions = test(model, test_loader)
np.save('./Test_results', predictions.numpy())